# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
import os, glob
from macrohet import dataio, tile, visualise, notify

from macrohet import visualise
import os
import re
import numpy as np
import cv2
import btrack

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
%%time
expt_ID = 'PS0000'

base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 11.6 s, sys: 915 ms, total: 12.5 s
Wall time: 12.7 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
4   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
5   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99
6   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99

#### Load images using Zarr (WIP)

In [4]:
import zarr

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [12]:
import numpy as np
from tqdm.auto import tqdm

# Make some glimpses

In [6]:
# Define the pattern to match the file names
pattern = re.compile(r'\[(.*?)\]')


In [7]:
unique_IDs = [pattern.search(filename).group(1).replace("'", "") for filename in glob.glob('/mnt/SYNO/macrohet_syno/manuscript/dt_dev/*pdf')]
# unique_IDs = 

In [14]:
import pandas as pd
PS_sc_df_fn = '/mnt/SYNO/macrohet_syno/PS0000/results/preliminary_sc_measures/sc_dfs/sc_df_GT_70_area_false_outliers_removed.pkl'

sc_df = df = pd.read_pickle(PS_sc_df_fn)
image_dir = os.path.join(base_dir, 'acquisition/Images')


In [8]:
segmentation=None
crop_size=side_length=500
track_scale_factor=5.04
mask_outline=True

In [10]:
# Sort the IDs based on their last two indices
sorted_ids = sorted(unique_IDs, key=lambda x: tuple(map(int, x.split('.')[-2:])))

In [24]:
str(acq_ID) in loaded_image 

True

In [25]:
images.shape

(75, 2, 1, 6048, 6048)

In [27]:
images = images[:,:,0,...]
loaded_image = str(acq_ID)

In [31]:
loaded_image = ''

In [32]:
%%time
for unique_ID in tqdm(sorted_ids, total = len(unique_IDs)):
    if os.path.exists(f"/mnt/SYNO/macrohet_syno/manuscript/dt_dev_glimpses/{unique_ID}.mp4"):
        continue
    acq_ID= row, column = int(unique_ID.split('.')[1]), int(unique_ID.split('.')[2]) 
    ID = int(unique_ID.split('.')[0])
    if str(acq_ID) not in loaded_image:
        # print(f'Loading images for {unique_ID}')
        # image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        # zarr_group = zarr.open(image_dir, mode='r')
        # images = (zarr_group.images[:,:,0,...])
        base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'
        loaded_image = str(acq_ID)
        image_dir = os.path.join(base_dir, 'acquisition/Images')
        images = tile.compile_mosaic(image_dir, 
                                     metadata, 
                                     row, column, 
                                     # subset_field_IDs=['16', '17',  '20', '21'], 
                                     # n_tile_rows = 2, n_tile_cols = 2,
                                     set_plane=1,
                                     # set_channel=1,
                                     # set_time = 1,
        #                             input_transforms = [input_transforms]
                                    ).compute().compute()
        images = images[:,:,0,...]
    rgb_stack = []
    # if segmentation is None:
    #     # load segmentation
    #     with btrack.io.HDF5FileHandler(os.path.join(base_dir,
    #                                                 f'labels/macrohet_seg_model/{acq_ID}.h5'),
    #                                    'r',
    #                                    obj_type='obj_type_1') as reader:
    #         segmentation = reader.segmentation
    
    # extract single cell df
    sc_df = df[df['Unique ID'] == unique_ID]
    
    time_values = sc_df['Time (hours)']
    for t in tqdm(time_values):
        t = int(t)
        sc_df_t = sc_df[sc_df['Time (hours)'] == t]
    
        # Extract xy coordinates and transpose for python and area from the cell information
        y_coord, x_coord, area, t = sc_df_t.loc[:, ['x', 'y', 'Mphi Area', 'Time (hours)']].values[0]
    
        # Scale according to tracking shrinkage
        y_coord, x_coord = y_coord * track_scale_factor, x_coord * track_scale_factor
    
        if not crop_size:
            # Calculate the side length for cropping based on the square root of the area
            side_length = int(np.sqrt(area)) * 2
    
        # Calculate the cropping boundaries
        x_start = int(x_coord - side_length / 2)
        x_end = int(x_coord + side_length / 2)
        y_start = int(y_coord - side_length / 2)
        y_end = int(y_coord + side_length / 2)
    
        # Pad the boundaries if they exceed the image dimensions
        if x_start < 0:
            x_pad = abs(x_start)
            x_start = 0
        else:
            x_pad = 0
    
        if x_end > images.shape[2]:
            x_pad_end = x_end - images.shape[2]
            x_end = images.shape[2]
        else:
            x_pad_end = 0
    
        if y_start < 0:
            y_pad = abs(y_start)
            y_start = 0
        else:
            y_pad = 0
    
        if y_end > images.shape[3]:
            y_pad_end = y_end - images.shape[3]
            y_end = images.shape[3]
        else:
            y_pad_end = 0
    
        # Crop the image
        cropped_image = images[int(t), :, x_start:x_end, y_start:y_end]
    
        # Pad the cropped image if necessary
        cropped_image = np.pad(cropped_image, ((0, 0), (x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
    
        # extract the gfp and rfp channels to apply some vis techn
        gfp = cropped_image[0, ...]#.compute().compute()
        rfp = cropped_image[1, ...]#.compute().compute()
    
        # clip the images so that the contrast is more apparent
        contrast_lim_gfp = np.clip(gfp, 358, 5886)
        contrast_lim_rfp = np.clip(rfp, 480, 1300)
    
        norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
    
        # put the modified gfp rfp back in place
        cropped_image[0, ...] = norm_gfp
        cropped_image[1, ...] = norm_rfp
    
        # Create an empty RGB image with the same shape as the input image
        rgb_image = np.zeros((cropped_image.shape[1], cropped_image.shape[2], 3), dtype=np.uint16)
    
        # Assign the first channel to the green channel of the RGB image
        rgb_image[:, :, 1] = cropped_image[0]
    
        # Assign the second channel to the red and blue channels of the RGB image to create magenta
        rgb_image[:, :, 0] = cropped_image[1]
        rgb_image[:, :, 2] = cropped_image[1]
    
        # scale down to 8bit
        rgb_image = np.uint8(rgb_image >> 8)
    
        # if mask_outline:
        #     # load mask (singular)
        #     cropped_masks = segmentation[int(t), x_start:x_end, y_start:y_end]
    
        #     # Pad the cropped image if necessary
        #     cropped_masks = np.pad(cropped_masks, ((x_pad, x_pad_end), (y_pad, y_pad_end)), mode='constant')
    
        #     # extract only that segment
        #     seg_ID = cropped_masks[int(cropped_masks.shape[0] / 2), int(cropped_masks.shape[1] / 2)]
        #     instance_mask = (cropped_masks == seg_ID).astype(np.uint8)
    
        #     # draw outline
        #     contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #     cv2.drawContours(rgb_image, contours, -1, (0, 2 ** 8, 2 ** 8), thickness=2)  # make 8bit
    
        # downsize image to reduce storage demands
        rgb_image = cv2.resize(rgb_image, (rgb_image.shape[1] // 2, rgb_image.shape[0] // 2))
        rgb_stack.append(rgb_image)
    
    max_height = max(rgb_image.shape[0] for rgb_image in rgb_stack)
    max_width = max(rgb_image.shape[1] for rgb_image in rgb_stack)
    
    # Resize each image to the maximum dimensions
    resized_rgb_stack = [cv2.resize(rgb_image, (max_width, max_height)) for rgb_image in rgb_stack]
    
    # Stack the resized images
    rgb_stack = np.stack(resized_rgb_stack, axis=0)
    
    # Get the dimensions of the first frame
    height, width, _ = rgb_stack[0].shape
    
    # Define the frame rate (number of frames per second)
    frame_rate = len(rgb_stack) / 20  # Total frames divided by total seconds
    output_file = f"/mnt/SYNO/macrohet_syno/manuscript/dt_dev_glimpses/{unique_ID}.mp4"
    # Initialize VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))
    
    # Write each frame to the video file
    for frame in rgb_stack:
        out.write(frame)
    
    # Release the VideoWriter object
    out.release()
    
    print(f"Video {unique_ID} saved successfully.")

  0%|          | 0/672 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

Video 159.5.6 saved successfully.


  0%|          | 0/75 [00:00<?, ?it/s]

Video 207.5.6 saved successfully.


...

Video 312.6.9 saved successfully.


  0%|          | 0/75 [00:00<?, ?it/s]

Video 365.6.9 saved successfully.


  0%|          | 0/73 [00:00<?, ?it/s]

Video 740.6.9 saved successfully.
CPU times: user 12h 10min 1s, sys: 1h 5min 3s, total: 13h 15min 5s
Wall time: 52min 36s


In [249]:
viewer = napari.Viewer()

viewer.add_image(rgb_stack)

<Image layer 'rgb_stack' at 0x7f05e7ff61f0>

In [250]:
rgb_stack.shape

(75, 191, 191, 3)

In [203]:
from macrohet import notify

In [204]:
notify.send_sys_message()

In [18]:
import btrack

In [96]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/{expt_ID}/labels/macrohet_seg_model/{row, column}.h5'), 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation
                tracks = reader.tracks

[INFO][2024/02/06 04:46:19 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/PS0000/labels/macrohet_seg_model/(3, 5).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/PS0000/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2024/02/06 04:46:29 pm] Loading segmentation (75, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (75, 6048, 6048)
[INFO][2024/02/06 04:46:29 pm] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2024/02/06 04:46:29 pm] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2024/02/06 04:46:29 pm] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2024/02/06 04:46:30 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/PS0000/labels/macrohet_seg_model/(3, 5).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/PS0000/labels/macrohet_seg_model/(3, 5).h5


In [97]:
viewer.add_labels(segmentation,)

<Labels layer 'segmentation' at 0x7f07748ff490>

In [158]:
[t for t in tracks if t.ID == 1][0]

,ID,t,x,y,z,parent,root,state,generation,dummy,minor_axis_length,major_axis_length,Infected,area,orientation,mean_intensity
0,1,0,519.922607,876.779602,0.000000,1,1,5,0,False,139.338257,272.300110,1.0,29567.0,-1.259609,"(3,) array"
1,1,1,522.290833,876.766357,0.000000,1,1,5,0,False,133.214874,251.294281,1.0,26181.0,-1.247703,"(3,) array"
2,1,2,524.336243,874.563110,0.000000,1,1,5,0,False,130.527573,256.213470,1.0,26043.0,-1.220798,"(3,) array"
3,1,3,516.952454,876.656799,0.000000,1,1,5,0,False,151.449524,221.050552,1.0,25605.0,-1.213046,"(3,) array"
4,1,4,521.947449,880.909363,0.000000,1,1,5,0,False,170.013687,200.579819,1.0,26430.0,1.540157,"(3,) array"
5,1,5,523.277527,878.426208,0.000000,1,1,5,0,False,124.448059,217.377197,1.0,21047.0,-1.195608,"(3,) array"
6,1,6,521.043518,877.680420,0.000000,1,1,5,0,False,110.176552,147.966309,1.0,12742.0,-1.193765,"(3,) array"
7,1,7,522.131592,878.283936,0.000000,1,1,5,0,False,112.954071,165.080383,1.0,14611.0,-1.565078,"(3,) array"
8,1,8,518.135559,877.770569,0.000000,1,1,5,0,False,123.692322,160.646317,1.0,15471.0,-1.518471,"(3,) array"
9,1,9,516.401733,876.479553,-1.314526,1,1,5,0,True,NaN,NaN,NaN,NaN,NaN,"(3,) array"


In [159]:
viewer.add_points((876*5.04, 519*5.04), size=100)

<Points layer 'Points [3]' at 0x7f0785e82970>

In [105]:
frame = segmentation[0]

In [161]:
mask = frame == 423

In [162]:
rfp_frame = images

In [163]:
masked_image = rfp_frame * mask


In [164]:
threshold = 480

In [165]:
viewer.add_image(masked_image)

<Image layer 'masked_image [2]' at 0x7f075b16cc10>

In [166]:
# Apply threshold to identify mtb region
thresholded_masked_image = masked_image >= threshold



In [167]:
# Calculate the area of the microtubule region
mtb_area = np.sum(thresholded_masked_image)

In [173]:
viewer.add_image(thresholded_masked_image)

<Image layer 'thresholded_masked_image [1]' at 0x7f05e6cbd6d0>

In [175]:
np.sum(thresholded_masked_image)

5471

In [177]:
from macrohet import tools 

In [178]:
track = [t for t in tracks if t.ID ==1][0]

In [191]:
del fake_stack

NameError: name 'fake_stack' is not defined

In [179]:
help(tools.measure_mtb_area)

Help on function measure_mtb_area in module macrohet.tools:

measure_mtb_area(track, masks, rfp_images, threshold=480, scale_factor=5.04, image_resolution=1.4949402023919043e-07)
    Measures the area of a region in each frame of an image sequence.
    
    Parameters:
    - track (object): An object containing tracking information with attributes 't', 'x', 'y', and 'ID'.
    - masks (array): A numpy array representing the segmented masks of the images.
    - rfp_images (array): A numpy array of RFP (red fluorescent protein) images.
    - threshold (int): The intensity threshold for considering a pixel as part of the region.
    - scale_factor (float): Factor for scaling the coordinates from the track object.
    - image_resolution (float): The resolution of the images in pixels per meter.
    
    Returns:
    - mtb_areas (list): A list of areas (in micrometers squared) of the region for each frame.
    
    The function iterates over each frame specified in the track object, scales t

In [185]:
from tqdm.auto import tqdm

In [186]:
fake_stack = np.stack([rfp_frame for frame in tqdm(range(75))], axis = 0)

  0%|          | 0/75 [00:00<?, ?it/s]

In [187]:
mtb_values = tools.measure_mtb_area(track, segmentation, fake_stack)

Calculating mtb area for every frame in track: 1:   0%|          | 0/73 [00:00<?, ?it/s]

In [189]:
mtb_values[0]

122.26843607948412

In [168]:
print(mtb_area)

5471


In [169]:
 image_resolution=1.4949402023919043e-07

In [170]:
# Convert the resolution to pixels per micrometer
resolution_micrometers_per_pixel = image_resolution * 1_000_000

# Convert area from pixels to micrometers squared
mtb_area = mtb_area * (resolution_micrometers_per_pixel ** 2)

In [171]:
(resolution_micrometers_per_pixel ** 2)

0.022348462087275475

In [172]:
mtb_area

122.26843607948412

In [157]:
mtb_area/2.368937

2.23584903890829

In [129]:
manual_mask = viewer.layers['Labels'].data[0]

In [130]:
manual_mask.shape

(6048, 6048)

In [131]:
np.sum(manual_mask)

5817

In [25]:
# Select the corresponding frame from the masks
frame = masks[t, ...]

# Check to see if mask exists at the specified coordinates
if frame[y, x]:
    # Select the specific cell of interest based on the mask
    mask = frame == frame[y, x]

    # Apply mask to the corresponding RFP frame
    rfp_frame = rfp_images[t]
    masked_image = rfp_frame * mask

    # Apply threshold to identify mtb region
    thresholded_masked_image = masked_image >= threshold

    # Calculate the area of the microtubule region
    mtb_area = np.sum(thresholded_masked_image)

    # Convert the resolution to pixels per micrometer
    resolution_micrometers_per_pixel = image_resolution * 1_000_000

    # Convert area from pixels to micrometers squared
    mtb_area = mtb_area * (resolution_micrometers_per_pixel ** 2)

    # Append the calculated area to the list
    mtb_areas.append(mtb_area)

<Labels layer 'segmentation' at 0x7fed26e6c0d0>

# Previous ND2

In [91]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
zarr_group = zarr.open(image_dir, mode='r')

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [9]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'acquisition/Images')
dask_images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # subset_field_IDs=['16', '17',  '20', '21'], 
                             # n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             # set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()
dask_images

CPU times: user 658 ms, sys: 3.96 ms, total: 662 ms
Wall time: 670 ms


dask.array<max-aggregate, shape=(75, 2, 6048, 6048), dtype=uint16, chunksize=(1, 2, 2160, 2160), chunktype=numpy.ndarray>

In [11]:
images = zarr_group.images[0,0,...]
images.shape

(3, 6048, 6048)

In [13]:
import numpy as np

In [14]:
images = np.sum(images, axis = 0)

In [15]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}')

viewer.add_image(images,)
                 # channel_axis = 1, 
                 # # scale = napari_scale, 
                 # contrast_limits=[[280, 1000],[0,3000]])

<Image layer 'images' at 0x7efe45a439a0>

In [23]:
%%time
z1 = images[:,:,0,...]

CPU times: user 18.4 s, sys: 59.3 s, total: 1min 17s
Wall time: 3min 46s


In [18]:
import btrack

In [19]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/{expt_ID}/labels/macrohet_seg_model/{row, column}_first_pass.h5'), 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation
                # tracks = reader.tracks

[INFO][2024/02/06 03:13:25 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 3)_first_pass.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 3)_first_pass.h5...
[INFO][2024/02/06 03:13:44 pm] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/02/06 03:13:44 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 3)_first_pass.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 3)_first_pass.h5


In [20]:
viewer.add_labels(segmentation,)

<Labels layer 'segmentation' at 0x7efd8dabbe80>

In [27]:
x_coords = [int(xi) for xi in [5472.746246039399,
 5472.746246039399,
 5486.096242254647,
 5500.329572263822,
 5510.675596905509,
 5438.693065970174,
 5480.530829671062,
 5461.236876504988,
 5469.6922642972995,
 5474.665007505728,
 5474.372622724507,
 5441.25071849947,
 5431.163245356794,
 5447.638106650689,
 5401.097859501309,
 5458.2037895123185,
 5423.897505528184,
 5450.797177441847,
 5393.217742272784,
 5411.22176165803,
 5416.824555429435,
 5383.79879462216,
 5413.5038321727325,
 5393.169633749054,
 5373.554017938861,
 5368.331506843434,
 5430.211934875269,
 5381.755155770074,
 5392.150312435623,
 5392.317724801933,
 5374.908793911906,
 5390.2561899213515,
 5379.819480780841,
 5390.4406485903555,
 5377.032280935897,
 5371.854852826017,
 5405.003818615752,
 5405.07407552645,
 5376.475064387731,
 5390.252700798497,
 5351.455846406826,
 5377.990615856705,
 5391.0602309372725,
 5361.77590530645,
 5389.48476871004,
 5386.070360673001,
 5310.099368625545,
 5285.215779825395,
 5260.332191025245,
 5235.448602225095,
 5205.790368271954,
 5198.133702975856,
 5179.958051782966,
 5163.646741327221,
 5147.335430871476,
 5220.276700434152,
 5202.920935330654,
 5231.698149715806,
 5211.194877699849,
 5213.240591151092,
 5223.266544335267,
 5227.718656605586,
 5232.170768875904,
 5236.622881146222,
 5241.0749934165415,
 5173.338289859115,
 5170.376619479919,
 5167.414949100722,
 5164.453278721526,
 5240.135933745719,
 5228.750556912424,
 5245.063622820085,
 5207.907769862495,
 5260.609250112258,
 5326.639073867809,
 5349.205111306505,
 5376.345633825899,
 5410.764567821455,
 5369.3243895167625,
 5359.88331446765,
 5329.831735933239,
 5319.949738754902,
 5388.253754230118,
 5424.675502518108,
 5397.988879274428,
 5434.800431358973,
 5471.611983443518,
 5508.4235355280625,
 5544.107951509484,
 5551.395430473596,
 5550.066381082342,
 5548.452303643447,
 5549.155080029634,
 5561.186685412471,
 5567.791950712697,
 5557.699342445248,
 5557.823387269993,
 5558.007319352159,
 5560.183715182935,
 5546.057975010771,
 5550.868006259876,
 5555.678037508981,
 5557.046358524208,
 5553.236282307818,
 5549.426206091429,
 5551.121324224907,
 5557.426843742876,
 5553.940335205612,
 5553.498974602256,
 5549.4003826054495,
 5544.361776162036,
 5492.424959871588,
 5492.424959871588,
 5402.656685734716,
 5373.853899728852,
 5369.858697374676,
 5371.848683878138,
 5373.860857176777,
 5365.598916852501,
 5363.3692751718145,
 5361.139633491127,
 5358.90999181044,
 5356.680350129753,
 5371.074742777998,
 5380.688404950957,
 5414.278243208936,
 5398.956768986181,
 5389.825679978848,
 5380.694590971516,
 5320.5197714285705,
 5366.895837873175,
 5366.895837873175,
 5367.842764753435,
 5373.271641791045,
 5369.232279495761,
 5387.294447560292,
 5411.240868057632,
 5399.933467336684]]

In [26]:
y_coords = [int(xi) for xi in [191.1370712219314,
 191.1370712219314,
 176.7415550669598,
 177.87167914657914,
 179.62931063593308,
 159.53955682871342,
 140.08889332800638,
 142.64451324389404,
 119.53324871306678,
 106.82223275657739,
 99.73373315589187,
 128.2833913175011,
 140.42372794155477,
 161.27786099460755,
 222.99925993396334,
 174.0992797173923,
 172.9274935019591,
 161.60818876554913,
 205.59324940399014,
 162.8885334534805,
 202.30953461975028,
 216.17217645590387,
 193.67551351114614,
 215.42851031623286,
 228.06040637012632,
 228.67406285768715,
 192.86506882817412,
 232.19053566003984,
 204.6973837808144,
 207.60944327992325,
 209.83373049427317,
 188.7638362947859,
 182.99219158784464,
 180.22003543367737,
 204.85189973724817,
 194.98424616518494,
 179.14793158313444,
 196.07419316889533,
 219.35397643018806,
 187.26384559545667,
 216.75721079063152,
 213.6554627845572,
 167.37204826797043,
 217.32174741699268,
 179.59289206468597,
 176.6345965743065,
 188.62671199611458,
 166.31634045849256,
 144.00596892087054,
 121.69559738324854,
 55.39108011912544,
 58.18816676024704,
 40.3980062888874,
 24.37581387428953,
 8.35362145969166,
 43.919247467438495,
 48.26949214468395,
 67.73374450759866,
 97.71024808960534,
 125.01220855516797,
 132.79794601633716,
 143.92312942545448,
 155.04831283457176,
 166.17349624368907,
 177.29867965280638,
 196.6720323615567,
 212.43817694641277,
 228.20432153126882,
 243.97046611612487,
 232.906702111344,
 223.76657988673875,
 203.4379765138901,
 215.9927579106809,
 196.31953300404132,
 186.89895016233382,
 205.2625718766837,
 225.94766600202124,
 256.01724866433636,
 188.86383716833345,
 173.76541801915985,
 191.20998925886144,
 178.73719570397313,
 233.2043675972927,
 266.42189875207697,
 328.1910713713425,
 366.5394414776547,
 404.88781158396694,
 443.2361816902791,
 472.3920818861875,
 490.6148433953699,
 469.9704454770006,
 466.6909563196391,
 455.62502369015004,
 442.6712391230003,
 418.6171120265393,
 428.84796051107503,
 427.82691343882004,
 413.25553709856035,
 355.02471338053033,
 382.6373632055148,
 383.5781971575975,
 384.5190311096802,
 352.77941493426334,
 342.17885284520594,
 331.57829075614853,
 374.90753547031005,
 377.7983091086394,
 366.92456745843765,
 362.7705244329434,
 411.162420679358,
 388.93939236561926,
 257.54574638844304,
 257.54574638844304,
 279.8389345727565,
 358.6855760540167,
 410.2059056573873,
 406.9155762138129,
 395.71221654992286,
 411.70356801529147,
 420.2770395907584,
 428.85051116622543,
 437.4239827416924,
 445.9974543171594,
 394.71413731697663,
 358.1511560018683,
 257.45715917745616,
 304.4553014553014,
 342.063924823419,
 379.67254819153646,
 341.3584,
 429.78562867727175,
 429.78562867727175,
 410.51114447395776,
 366.2247416762342,
 338.7122511211395,
 325.2351654514862,
 244.36659293358045,
 172.72020100502513]]

In [32]:
xy_coords = [(ti, xi, yi )for (ti, xi, yi) in zip(range(0, len(y_coords)), y_coords, x_coords)]

In [33]:
xy_coords

[(0, 191, 5472),
 (1, 191, 5472),
 (2, 176, 5486),
 (3, 177, 5500),
 (4, 179, 5510),
 (5, 159, 5438),
 (6, 140, 5480),
 (7, 142, 5461),
 (8, 119, 5469),
 (9, 106, 5474),
 (10, 99, 5474),
 (11, 128, 5441),
 (12, 140, 5431),
 (13, 161, 5447),
 (14, 222, 5401),
 (15, 174, 5458),
 (16, 172, 5423),
 (17, 161, 5450),
 (18, 205, 5393),
 (19, 162, 5411),
 (20, 202, 5416),
 (21, 216, 5383),
 (22, 193, 5413),
 (23, 215, 5393),
 (24, 228, 5373),
 (25, 228, 5368),
 (26, 192, 5430),
 (27, 232, 5381),
 (28, 204, 5392),
 (29, 207, 5392),
 (30, 209, 5374),
 (31, 188, 5390),
 (32, 182, 5379),
 (33, 180, 5390),
 (34, 204, 5377),
 (35, 194, 5371),
 (36, 179, 5405),
 (37, 196, 5405),
 (38, 219, 5376),
 (39, 187, 5390),
 (40, 216, 5351),
 (41, 213, 5377),
 (42, 167, 5391),
 (43, 217, 5361),
 (44, 179, 5389),
 (45, 176, 5386),
 (46, 188, 5310),
 (47, 166, 5285),
 (48, 144, 5260),
 (49, 121, 5235),
 (50, 55, 5205),
 (51, 58, 5198),
 (52, 40, 5179),
 (53, 24, 5163),
 (54, 8, 5147),
 (55, 43, 5220),
 (56, 48, 

In [34]:
viewer.add_points(xy_coords, size=10)

<Points layer 'Points [6]' at 0x7f076c4ec640>

In [51]:
frame = segmentation[0]

In [52]:
mask = frame == 21

In [53]:
rfp_frame = images

In [54]:
masked_image = rfp_frame * mask


In [55]:
threshold = 480

In [72]:
# Apply threshold to identify mtb region
thresholded_masked_image = masked_image >= threshold



In [73]:
# Calculate the area of the microtubule region
mtb_area = np.sum(thresholded_masked_image)

In [74]:
print(mtb_area)

439


In [75]:
 image_resolution=1.4949402023919043e-07

In [76]:
# Convert the resolution to pixels per micrometer
resolution_micrometers_per_pixel = image_resolution * 1_000_000

# Convert area from pixels to micrometers squared
mtb_area = mtb_area * (resolution_micrometers_per_pixel ** 2)

In [77]:
(resolution_micrometers_per_pixel ** 2)

0.022348462087275475

In [78]:
mtb_area

9.810974856313933

In [84]:
manual_mask = viewer.layers['Labels'].data[0]

In [85]:
manual_mask.shape

(6048, 6048)

In [87]:
np.sum(manual_mask)/2

435.0

In [25]:
# Select the corresponding frame from the masks
frame = masks[t, ...]

# Check to see if mask exists at the specified coordinates
if frame[y, x]:
    # Select the specific cell of interest based on the mask
    mask = frame == frame[y, x]

    # Apply mask to the corresponding RFP frame
    rfp_frame = rfp_images[t]
    masked_image = rfp_frame * mask

    # Apply threshold to identify mtb region
    thresholded_masked_image = masked_image >= threshold

    # Calculate the area of the microtubule region
    mtb_area = np.sum(thresholded_masked_image)

    # Convert the resolution to pixels per micrometer
    resolution_micrometers_per_pixel = image_resolution * 1_000_000

    # Convert area from pixels to micrometers squared
    mtb_area = mtb_area * (resolution_micrometers_per_pixel ** 2)

    # Append the calculated area to the list
    mtb_areas.append(mtb_area)

<Labels layer 'segmentation' at 0x7fed26e6c0d0>